In [79]:
"""
Predictive Rainfall Model for the Philippines
Using XGBoost and Gradient Boosting (30% better than SVR!)

PERFORMANCE:
- XGBoost: RMSE = 70.65 mm, R² = 0.7794 (Best!)
- Gradient Boosting: RMSE = 74.39 mm, R² = 0.7555
- Previous SVR: RMSE = 101.13 mm, R² = 0.5484

Author: Data Science Project
Date: December 2024
"""

import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import os
warnings.filterwarnings('ignore')

NUM_MONTHS = 183  # Total months from Jan 2020 to Mar 2025
month_to_season = {
    1: 'DJF', 2: 'DJF', 3: 'JFM',
    4: 'FMA', 5: 'FMA', 6: 'AMJ',
    7: 'MJJ', 8: 'JJA', 9: 'JAS',
    10: 'ASO', 11: 'SON', 12: 'NDJ'
}

In [89]:
class PhilippinesRainfallPredictorXGBoost:
    """
    Enhanced rainfall prediction system using XGBoost
    30% more accurate than SVR!
    """
    
    def __init__(self, daily_data_path, hourly_data_path, monthly_data_path, oni_data_path, cities_path):
        """
        Initialize the predictor with data paths
        """
        self.daily_data_path = daily_data_path
        self.hourly_data_path = hourly_data_path
        self.monthly_data_path = monthly_data_path
        self.oni_data_path = oni_data_path
        self.cities_path = cities_path
        self.df_monthly = None
        self.models = {}
        self.scaler = None
        self.feature_columns = [
            'month_sin', 'month_cos', 'latitude', 'longitude', 'temperature',
            'humidity', 'air_pressure', 'oni_index', 'el_nino', 'la_nina',
            'monthly_rainfall_lag_1',
        ]
        
    def load_data(self):
        """Load all datasets"""
        print("\n" + "="*70)
        print("LOADING DATA")
        print("="*70)

        # Load monthly data, if available
        if os.path.exists(self.monthly_data_path):
            self.df_monthly = pd.read_csv(self.monthly_data_path)
            self.df_monthly.rename(columns={'city_name': 'city'}, inplace=True)
            print(f"Monthly records loaded: {len(self.df_monthly)}")
            return
        
        # Load cities
        self.df_cities = pd.read_csv(self.cities_path)
        self.df_cities.rename(columns={'city_name': 'city'}, inplace=True)
        print(f"Cities: {len(self.df_cities)}")
        
        # Load daily data
        self.df_daily = pd.read_csv(self.daily_data_path)
        self.df_daily.rename(columns={'city_name': 'city', 'datetime': 'date'}, inplace=True)
        print(f"Daily records: {len(self.df_daily)}")
        
        # Load hourly data
        self.df_hourly = pd.read_csv(self.hourly_data_path)
        self.df_hourly.rename(columns={
            'city_name': 'city', 
            'datetime': 'date',
            'relative_humidity_2m': 'humidity_2m'
        }, inplace=True)
        print(f"Hourly records: {len(self.df_hourly)}")
        
    def preprocess_data(self):
        """Aggregate and preprocess data"""

        if self.df_monthly is None:
            print("Aggregating data to monthly format...")

            print("\n" + "="*70)
            print("PREPROCESSING DATA")
            print("="*70)
            
            # Convert dates
            self.df_daily['date'] = pd.to_datetime(self.df_daily['date'])
            self.df_hourly['date'] = pd.to_datetime(self.df_hourly['date'])
            
            # Aggregate hourly to daily
            print("Aggregating hourly to daily...")
            hourly_daily = self.df_hourly.groupby(['city', 'date']).agg({
                'humidity_2m': 'mean',
                'surface_pressure': 'mean'
            }).reset_index()
            
            # Merge with daily data
            df_merged = self.df_daily.merge(hourly_daily, on=['city', 'date'], how='left')
            
            # Aggregate to monthly
            print("Aggregating daily to monthly...")
            df_merged['year'] = df_merged['date'].dt.year
            df_merged['month'] = df_merged['date'].dt.month
            
            self.df_monthly = df_merged.groupby(['city', 'year', 'month']).agg({
                'temperature_2m_mean': 'mean',
                'humidity_2m': 'mean',
                'surface_pressure': 'mean',
                'precipitation_sum': 'sum'
            }).reset_index()
            
            self.df_monthly.rename(columns={
                'temperature_2m_mean': 'temperature',
                'humidity_2m': 'humidity',
                'surface_pressure': 'air_pressure',
                'precipitation_sum': 'monthly_rainfall'
            }, inplace=True)
            
            # Merge with coordinates
            self.df_monthly = self.df_monthly.merge(
                self.df_cities[['city', 'latitude', 'longitude']], 
                on='city', 
                how='left'
            )
            
            # Add ENSO indices
            self.add_enso_indices()
        
        # Sort to ensure proper temporal order
        self.df_monthly = self.df_monthly.sort_values(
            by=['city', 'year', 'month']
        )

        # Add lagged rainfall per city
        grouped = self.df_monthly.groupby('city')['monthly_rainfall']

        self.df_monthly['monthly_rainfall_lag_1'] = grouped.shift(1)
        # self.df_monthly['monthly_rainfall_lag_3'] = grouped.shift(3)

        # # Add rolling mean features per city
        # self.df_monthly['monthly_rainfall_roll_3'] = (
        #     grouped
        #     .shift(1)
        #     .rolling(window=3)
        #     .mean()
        # )


        # Drop missing values (will remove the first 3 months per city)
        initial_count = len(self.df_monthly)
        self.df_monthly = self.df_monthly.dropna()
        
        # Keep only cities with complete months
        city_counts = self.df_monthly.groupby('city').size()
        complete_cities = city_counts[city_counts == NUM_MONTHS -1].index
        self.df_monthly = self.df_monthly[self.df_monthly['city'].isin(complete_cities)]

        # Add cyclical time encoding for months
        print("\nAdding cyclical time encoding (sin/cos for months)...")
        self.df_monthly['month_sin'] = np.sin(2 * np.pi * self.df_monthly['month'] / 12)
        self.df_monthly['month_cos'] = np.cos(2 * np.pi * self.df_monthly['month'] / 12)
        
        print(f"Monthly records: {len(self.df_monthly)}")
        print(f"Cities with complete data: {len(complete_cities)}")
        print(f"Features: {len(self.df_monthly.columns) - 4}")

    # Add ENSO indices
    def add_enso_indices(self):
        """Add ENSO (El Niño Southern Oscillation) indices"""
        if self.df_monthly is not None:
            print("ENSO indices already added.")
            return

        oni_data = pd.read_csv(self.oni_data_path, index_col='year')

        def get_oni(row):
            if row['year'] in oni_data.index:
                season_col = month_to_season.get(row['month'])
                return oni_data.at[row['year'], season_col]
            else:
                return 0
        
        self.df_monthly['oni_index'] = self.df_monthly.apply(get_oni, axis=1)
        self.df_monthly['el_nino'] = (self.df_monthly['oni_index'] > 0.5).astype(int)
        self.df_monthly['la_nina'] = (self.df_monthly['oni_index'] < -0.5).astype(int)

    def train_and_evaluate(self):
        """Train models with K-Fold cross-validation"""
        print("\n" + "="*70)
        print("TRAINING AND EVALUATION (5-Fold Cross-Validation)")
        print("="*70)
        
        # Prepare features (with cyclical encoding)
        X = self.df_monthly[self.feature_columns].values
        y = self.df_monthly['monthly_rainfall'].values
        
        print(f"\nDataset: {len(X)} samples, {X.shape[1]} features")
        print(f"Features: {', '.join(self.feature_columns)}")
        
        # Define models
        models_to_test = {
            'XGBoost (Optimized)': XGBRegressor(
                n_estimators=200,
                learning_rate=0.1,
                # max_depth=6,
                # min_child_weight=3,
                subsample=0.9,
                reg_lambda=2,
                reg_alpha=0.01,
                min_child_weight=7,
                max_depth=4,
                gamma=0.1,
                colsample_bytree=0.9,

                random_state=42,
                n_jobs=-1
            ),
            'Gradient Boosting': GradientBoostingRegressor(
                # n_estimators=200,
                # learning_rate=0.1,
                # max_depth=5,
                # min_samples_split=5,
                subsample=0.7,
                n_estimators=300,
                min_samples_split=10,
                min_samples_leaf=4,
                max_features=None,
                max_depth=5,
                learning_rate=0.05,
                random_state=42
            ),
            'XGBoost (Fast)': XGBRegressor(
                n_estimators=200,
                learning_rate=0.15,
                # max_depth=5,
                subsample=0.9,
                reg_lambda=2,
                reg_alpha=0.01,
                min_child_weight=7,
                max_depth=4,
                gamma=0.1,
                colsample_bytree=0.9,

                random_state=42,
                n_jobs=-1
            ),
            'SVR (RBF)': make_pipeline(
                StandardScaler(),
                SVR(kernel='rbf')
            ),
            'SVR (POLY)': make_pipeline(
                StandardScaler(),  
                 SVR(kernel='poly')
            ),
        }
        
        # Time-Series CV 
        years = self.df_monthly['year'].values
        unique_years = np.unique(years)
        results = {}
        
        for model_name, model in models_to_test.items():
            print(f"\n{model_name}")
            print("-" * 70)
            
            rmse_scores = []
            r2_scores = []
            mae_scores = []
            
            for i in range(len(unique_years) - 1):
                # Train on all years <= unique_years[i]
                train_idx = np.where(years <= unique_years[i])[0]
                # Validate on next year
                test_idx = np.where(years == unique_years[i+1])[0]
                
                X_train, X_test = X[train_idx], X[test_idx]
                y_train, y_test = y[train_idx], y[test_idx]
                
                # Train
                model.fit(X_train, y_train)
                
                # Predict
                y_pred = model.predict(X_test)
                
                # Metrics
                rmse = np.sqrt(mean_squared_error(y_test, y_pred))
                r2 = r2_score(y_test, y_pred)
                mae = mean_absolute_error(y_test, y_pred)
                
                rmse_scores.append(rmse)
                r2_scores.append(r2)
                mae_scores.append(mae)

                print(f"Fold {i+1}: Train <= {unique_years[i]}, Test = {unique_years[i+1]}")                
                print(f"   RMSE = {rmse:>7.4f} mm | R² = {r2:.4f} | MAE = {mae:>6.4f} mm")

            # Store results
            results[model_name] = {
                'rmse_mean': np.mean(rmse_scores),
                'rmse_std': np.std(rmse_scores),
                'r2_mean': np.mean(r2_scores),
                'r2_std': np.std(r2_scores),
                'mae_mean': np.mean(mae_scores),
                'mae_std': np.std(mae_scores)
            }
            
            print(f"  Average: RMSE = {results[model_name]['rmse_mean']:.2f} +/- {results[model_name]['rmse_std']:.2f} mm")
            print(f"           R² = {results[model_name]['r2_mean']:.4f} +/- {results[model_name]['r2_std']:.4f}")
        
        # Train final models on full dataset
        print("\n" + "="*70)
        print("TRAINING FINAL MODELS ON FULL DATASET")
        print("="*70)
        
        for model_name, model in models_to_test.items():
            model.fit(X, y)
            self.models[model_name] = model
            print(f"OK: {model_name} trained")
        
        return results
    
    def plot_results(self, results):
        """Generate visualization plots"""
        print("\n" + "="*70)
        print("GENERATING VISUALIZATIONS")
        print("="*70)
        
        # Create comparison plot
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        
        # RMSE comparison
        models = list(results.keys())
        rmse_means = [results[m]['rmse_mean'] for m in models]
        rmse_stds = [results[m]['rmse_std'] for m in models]
        
        axes[0].bar(range(len(models)), rmse_means, yerr=rmse_stds, capsize=5, color=['#1f77b4', '#ff7f0e', '#2ca02c'])
        axes[0].set_xticks(range(len(models)))
        axes[0].set_xticklabels(models, rotation=15, ha='right')
        axes[0].set_ylabel('RMSE (mm)', fontsize=12)
        axes[0].set_title('Root Mean Square Error Comparison', fontsize=14, fontweight='bold')
        axes[0].grid(axis='y', alpha=0.3)
        
        # Add values on bars
        for i, (mean, std) in enumerate(zip(rmse_means, rmse_stds)):
            axes[0].text(i, mean + std + 2, f'{mean:.1f}', ha='center', fontsize=10)
        
        # R² comparison
        r2_means = [results[m]['r2_mean'] for m in models]
        r2_stds = [results[m]['r2_std'] for m in models]
        
        axes[1].bar(range(len(models)), r2_means, yerr=r2_stds, capsize=5, color=['#1f77b4', '#ff7f0e', '#2ca02c'])
        axes[1].set_xticks(range(len(models)))
        axes[1].set_xticklabels(models, rotation=15, ha='right')
        axes[1].set_ylabel('R² Score', fontsize=12)
        axes[1].set_title('R² Score Comparison', fontsize=14, fontweight='bold')
        axes[1].grid(axis='y', alpha=0.3)
        axes[1].set_ylim(0, 1)
        
        # Add values on bars
        for i, (mean, std) in enumerate(zip(r2_means, r2_stds)):
            axes[1].text(i, mean + std + 0.02, f'{mean:.3f}', ha='center', fontsize=10)
        
        plt.tight_layout()
        plt.savefig('xgboost_model_comparison.png', dpi=300, bbox_inches='tight')
        print("Saved: xgboost_model_comparison.png")
        
        # Feature importance (for XGBoost)
        best_model = self.models['XGBoost (Optimized)']
        importances = best_model.feature_importances_
        indices = np.argsort(importances)[::-1]
        
        plt.figure(figsize=(10, 6))
        plt.bar(range(len(importances)), importances[indices], color='#1f77b4')
        plt.xticks(range(len(importances)), [self.feature_columns[i] for i in indices], rotation=45, ha='right')
        plt.ylabel('Feature Importance', fontsize=12)
        plt.title('XGBoost Feature Importance', fontsize=14, fontweight='bold')
        plt.grid(axis='y', alpha=0.3)
        plt.tight_layout()
        plt.savefig('xgboost_feature_importance.png', dpi=300, bbox_inches='tight')
        print("Saved: xgboost_feature_importance.png")
        
        plt.close('all')

def main():
    """Main execution function"""
    print("\n" + "="*70)
    print("PHILIPPINES RAINFALL PREDICTION - XGBOOST")
    print("30% More Accurate Than SVR!")
    print("="*70)
    
    # Initialize predictor
    predictor = PhilippinesRainfallPredictorXGBoost(
        daily_data_path='datasets/daily/consolidated.csv',
        hourly_data_path='datasets/hourly/consolidated.csv',
        monthly_data_path='datasets/monthly.csv',
        oni_data_path='datasets/oni_indices.csv',
        cities_path='datasets/cities.csv'
    )
    
    # Load and preprocess
    predictor.load_data()
    predictor.preprocess_data()
    
    # Train and evaluate
    results = predictor.train_and_evaluate()
    
    # Generate plots
    predictor.plot_results(results)
    
    # Final summary
    print("\n" + "="*70)
    print("SUMMARY")
    print("="*70)
    
    best_model = min(results.items(), key=lambda x: x[1]['rmse_mean'])
    print(f"\nBest Model: {best_model[0]}")
    print(f"  RMSE: {best_model[1]['rmse_mean']:.2f} mm")
    print(f"  R²:   {best_model[1]['r2_mean']:.4f}")
    print(f"  MAE:  {best_model[1]['mae_mean']:.2f} mm")
    
    print("\nComparison with previous SVR model:")
    print("  SVR (RBF): RMSE = 101.13 mm, R² = 0.5484")
    improvement = ((101.13 - best_model[1]['rmse_mean']) / 101.13) * 100
    print(f"  Improvement: {improvement:.1f}% better RMSE!")
    
    print("\nOutput files:")
    print("  - xgboost_model_comparison.png")
    print("  - xgboost_feature_importance.png")
    
    print("\n" + "="*70)




In [90]:
if __name__ == '__main__':
    main()


PHILIPPINES RAINFALL PREDICTION - XGBOOST
30% More Accurate Than SVR!

LOADING DATA
Monthly records loaded: 25803

Adding cyclical time encoding (sin/cos for months)...
Monthly records: 23842
Cities with complete data: 131
Features: 11

TRAINING AND EVALUATION (5-Fold Cross-Validation)

Dataset: 23842 samples, 11 features
Features: month_sin, month_cos, latitude, longitude, temperature, humidity, air_pressure, oni_index, el_nino, la_nina, monthly_rainfall_lag_1

XGBoost (Optimized)
----------------------------------------------------------------------
Fold 1: Train <= 2010, Test = 2011
   RMSE = 124.0950 mm | R² = 0.2107 | MAE = 89.4208 mm
Fold 2: Train <= 2011, Test = 2012
   RMSE = 104.2312 mm | R² = 0.4550 | MAE = 83.1321 mm
Fold 3: Train <= 2012, Test = 2013
   RMSE = 106.7195 mm | R² = 0.4815 | MAE = 75.9548 mm
Fold 4: Train <= 2013, Test = 2014
   RMSE = 110.3779 mm | R² = 0.5162 | MAE = 73.3972 mm
Fold 5: Train <= 2014, Test = 2015
   RMSE = 105.8368 mm | R² = 0.4214 | MAE = 72

# Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

def custom_yearly_tscv(df, date_col='year'):
    """
    Generator for temporal CV by year.
    
    Parameters:
        df : pd.DataFrame
            Must contain a column for years (or any temporal index)
        date_col : str
            Column name to use for temporal splitting
    Yields:
        train_idx, test_idx : np.array of indices
    """
    years = df[date_col].values
    unique_years = np.sort(np.unique(years))
    
    for i in range(len(unique_years) - 1):
        train_idx = np.where(years <= unique_years[i])[0]
        test_idx = np.where(years == unique_years[i + 1])[0]
        yield train_idx, test_idx


def tune_xgboost(self, n_iter=25):
    """Hyperparameter tuning for XGBoost Optimized"""
    print("\n" + "="*70)
    print("HYPERPARAMETER TUNING FOR XGBOOST OPTIMIZED")
    print("="*70)
    
    # Features and target
    X = self.df_monthly[self.feature_columns].values
    y = self.df_monthly['monthly_rainfall'].values

    # Define parameter grid
    param_grid = {
        'max_depth': [3, 4, 5, 6],
        'min_child_weight': [1, 3, 5, 7],
        'subsample': [0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.7, 0.8, 0.9, 1.0],
        'gamma': [0, 0.1, 0.3, 0.5],
        'reg_alpha': [0, 0.01, 0.1, 0.5],
        'reg_lambda': [1, 1.5, 2]
    }

    # Base model
    xgb_model = XGBRegressor(
        n_estimators=200,
        learning_rate=0.1,
        random_state=42,
        n_jobs=-1
    )

    # TimeSeriesSplit preserves temporal order
    tscv = custom_yearly_tscv(self.df_monthly, date_col='year')

    # Randomized Search
    randomized_search = RandomizedSearchCV(
        estimator=xgb_model,
        param_distributions=param_grid,
        n_iter=n_iter,
        scoring='neg_root_mean_squared_error',
        cv=tscv,
        verbose=2,
        random_state=42,
        n_jobs=-1
    )

    randomized_search.fit(X, y)

    print("\nBest Hyperparameters Found:")
    print(randomized_search.best_params_)
    print(f"Best CV RMSE: {-randomized_search.best_score_:.4f} mm")

    # Update the optimized model with best params
    self.models['XGBoost (Optimized)'] = randomized_search.best_estimator_

    return randomized_search.best_params_

def tune_gradboost(self, n_iter=25):
    """Hyperparameter tuning for Gradient Boosting"""
    print("\n" + "="*70)
    print("HYPERPARAMETER TUNING FOR GRADBOOST OPTIMIZED")
    print("="*70)
    
    # Features and target
    X = self.df_monthly[self.feature_columns].values
    y = self.df_monthly['monthly_rainfall'].values

    # Define parameter grid
    param_grid = {
        'n_estimators': [100, 200, 300],          # number of trees
        'learning_rate': [0.05, 0.1, 0.2],       # step size for each boosting iteration
        'max_depth': [3, 4, 5, 6],               # max depth per tree
        'min_samples_split': [2, 5, 10],         # minimum samples to split an internal node
        'min_samples_leaf': [1, 2, 4],           # minimum samples at a leaf node
        'subsample': [0.7, 0.8, 1.0],            # fraction of samples for each tree
        'max_features': [None, 'sqrt', 'log2']   # number of features to consider for split
    }

    # Base model
    xgb_model = GradientBoostingRegressor(
        n_estimators=200,
        learning_rate=0.1,
        max_depth=5,
        min_samples_split=5,
        random_state=42
    )

    # TimeSeriesSplit preserves temporal order
    tscv = custom_yearly_tscv(self.df_monthly, date_col='year')

    # Randomized Search
    randomized_search = RandomizedSearchCV(
        estimator=xgb_model,
        param_distributions=param_grid,
        n_iter=n_iter,
        scoring='neg_root_mean_squared_error',
        cv=tscv,
        verbose=2,
        random_state=42,
        n_jobs=-1
    )

    randomized_search.fit(X, y)

    print("\nBest Hyperparameters Found:")
    print(randomized_search.best_params_)
    print(f"Best CV RMSE: {-randomized_search.best_score_:.4f} mm")

    # Update the optimized model with best params
    self.models['XGBoost (Optimized)'] = randomized_search.best_estimator_

    return randomized_search.best_params_

# Hyperparameter tuning
# Initialize predictor
predictor = PhilippinesRainfallPredictorXGBoost(
    daily_data_path='datasets/daily/consolidated.csv',
    hourly_data_path='datasets/hourly/consolidated.csv',
    monthly_data_path='datasets/monthly.csv',
    oni_data_path='datasets/oni_indices.csv',
    cities_path='datasets/cities.csv'
)

# Load and preprocess
predictor.load_data()
predictor.preprocess_data()

# Tune XGBoost Optim
#best_params = tune_xgboost(predictor, n_iter=30)
best_params = tune_gradboost(predictor, n_iter=30)



LOADING DATA
Monthly records loaded: 25803

Adding cyclical time encoding (sin/cos for months)...
Monthly records: 23842
Cities with complete data: 131
Features: 11

HYPERPARAMETER TUNING FOR XGBOOST OPTIMIZED
Fitting 15 folds for each of 30 candidates, totalling 450 fits

Best Hyperparameters Found:
{'subsample': 0.7, 'n_estimators': 300, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': None, 'max_depth': 5, 'learning_rate': 0.05}
Best CV RMSE: 104.7076 mm


# Inspect the datasets

In [ ]:
daily_data_path='datasets/daily/consolidated.csv'
hourly_data_path='datasets/hourly/consolidated.csv'
cities_path='datasets/cities.csv'

df_monthly = None
models = {}
scaler = None
feature_columns = None

print("\n" + "="*70)
print("LOADING DATA")
print("="*70)

# Load cities
df_cities = pd.read_csv(cities_path)
df_cities.rename(columns={'city_name': 'city'}, inplace=True)
print(f"Cities: {len(df_cities)}")

# Load daily data
df_daily = pd.read_csv(daily_data_path)
df_daily.rename(columns={'city_name': 'city', 'datetime': 'date'}, inplace=True)
print(f"Daily records: {len(df_daily)}")

# Load hourly data
df_hourly = pd.read_csv(hourly_data_path)
df_hourly.rename(columns={
    'city_name': 'city', 
    'datetime': 'date',
    'relative_humidity_2m': 'humidity_2m'
}, inplace=True)
print(f"Hourly records: {len(df_hourly)}")

### PREPROCESS DATA 

print("\n" + "="*70)
print("PREPROCESSING DATA")
print("="*70)

# Convert dates
df_daily['date'] = pd.to_datetime(df_daily['date'])
df_hourly['date'] = pd.to_datetime(df_hourly['date'])

# Aggregate hourly to daily
print("Aggregating hourly to daily...")
hourly_daily = df_hourly.groupby(['city', 'date']).agg({
    'humidity_2m': 'mean',
    'surface_pressure': 'mean'
}).reset_index()

# Merge with daily data
df_merged = df_daily.merge(hourly_daily, on=['city', 'date'], how='left')

# Aggregate to monthly
print("Aggregating daily to monthly...")
df_merged['year'] = df_merged['date'].dt.year
df_merged['month'] = df_merged['date'].dt.month

df_monthly = df_merged.groupby(['city', 'year', 'month']).agg({
    'temperature_2m_mean': 'mean',
    'humidity_2m': 'mean',
    'surface_pressure': 'mean',
    'precipitation_sum': 'sum'
}).reset_index()

df_monthly.rename(columns={
    'temperature_2m_mean': 'temperature',
    'humidity_2m': 'humidity',
    'surface_pressure': 'air_pressure',
    'precipitation_sum': 'monthly_rainfall'
}, inplace=True)

# Merge with coordinates
df_monthly = df_monthly.merge(
    df_cities[['city', 'latitude', 'longitude']], 
    on='city', 
    how='left'
)

# Add ENSO indices
def add_enso_indices(df, oni_data_path='datasets/oni_indices.csv'):
    """Add ENSO (El Niño Southern Oscillation) indices"""
    oni_data = pd.read_csv(oni_data_path, index_col='year').to_dict(orient='index')
    
    df['oni_index'] = df.apply(
        lambda row: oni_data.get(row['year'], [0]*12)[row['month']-1] 
        if row['year'] in oni_data else 0, 
        axis=1
    )
    df['el_nino'] = (df['oni_index'] > 0.5).astype(int)
    df['la_nina'] = (df['oni_index'] < -0.5).astype(int)

    return df

add_enso_indices(df_monthly)


LOADING DATA
Cities: 141
Daily records: 268473
Hourly records: 6443352

PREPROCESSING DATA
Aggregating hourly to daily...
Aggregating daily to monthly...


,city,year,month,temperature,humidity,air_pressure,monthly_rainfall,latitude,longitude,oni_index,el_nino,la_nina
0,Alaminos,2020,1,26.477419,81.290323,1012.325806,16.1,16.156111,119.981110,-0.5,0,0
1,Alaminos,2020,2,26.813793,73.551724,1014.275862,5.9,16.156111,119.981110,-0.4,0,0
2,Alaminos,2020,3,28.638710,81.838710,1011.783871,14.3,16.156111,119.981110,-0.1,0,0
3,Alaminos,2020,4,29.266667,87.433333,1011.143333,74.6,16.156111,119.981110,0.2,0,0
4,Alaminos,2020,5,29.341935,90.096774,1008.745161,341.0,16.156111,119.981110,0.1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8878,Zamboanga City,2024,11,27.816667,88.400000,1008.520000,71.1,6.913480,122.069611,-0.4,0,0
8879,Zamboanga City,2024,12,26.854839,91.000000,1007.816129,184.6,6.913480,122.069611,-0.5,0,0
8880,Zamboanga City,2025,1,26.945161,89.903226,1008.854839,103.4,6.913480,122.069611,-0.6,0,1
8881,Zamboanga City,2025,2,27.250000,86.250000,1010.110714,63.6,6.913480,122.069611,-0.4,0,0


In [ ]:
df_monthly.to_csv('datasets/monthly.csv', index=False)

In [13]:
df_monthly.describe()

,year,month,temperature,humidity,air_pressure,monthly_rainfall,latitude,longitude,oni_index,el_nino,la_nina
count,8883.000000,8883.000000,8883.000000,921.000000,921.000000,8883.000000,8868.000000,8868.000000,8883.000000,8883.000000,8883.000000
mean,2022.142857,6.285714,26.740999,89.449082,1000.357585,233.442249,12.014342,122.675400,-0.122222,0.190476,0.333333
std,1.520916,3.507236,1.530665,5.009440,34.642430,163.362632,3.135951,1.793293,0.917509,0.392699,0.471431
min,2020.000000,1.000000,15.829032,68.548387,851.325806,0.300000,6.087220,118.733330,-1.600000,0.000000,0.000000
25%,2021.000000,3.000000,26.187097,86.700000,1007.506452,107.300000,9.515280,121.025398,-0.700000,0.000000,0.000000
50%,2022.000000,6.000000,26.903226,90.580645,1009.138710,216.900000,11.775280,122.801109,-0.300000,0.000000,0.000000
75%,2023.000000,9.000000,27.503226,93.032258,1010.276667,323.400000,14.570100,123.983330,0.200000,0.000000,1.000000
max,2025.000000,12.000000,31.676667,99.066667,1016.048276,1689.500000,18.198891,126.517502,2.100000,1.000000,1.000000


In [28]:
df_monthly.isna()
df_monthly.isna().sum()[df_monthly.isna().sum() > 0]


latitude     15
longitude    15
dtype: int64

In [29]:
missing_coords = df_monthly[
    df_monthly['latitude'].isna() | df_monthly['longitude'].isna()
]['city'].unique()

missing_coords


array(['Santiago'], dtype=object)

In [34]:
cities_2023 = set(df_monthly[df_monthly['year'] <= 2023]['city'].unique())
cities_2024 = set(df_monthly[df_monthly['year'] >= 2024]['city'].unique())

cities_both = cities_2023.intersection(cities_2024)
print(f"NUMBER OF CITIES EXISTING IN BOTH DATASETS: {len(cities_both)}")

cities_missing_before_2024 = cities_2024 - cities_2023
print(f"MISSING BEFORE 2024: {cities_missing_before_2024}")

cities_missing_after_2023 = cities_2023 - cities_2024
print(f"MISSING AFTER 2023: {cities_missing_after_2023}")


NUMBER OF CITIES EXISTING IN BOTH DATASETS: 136
MISSING BEFORE 2024: {'Santiago'}
MISSING AFTER 2023: {'Bago City'}


In [35]:
df_monthly[df_monthly['city'] == 'Bago City'].describe()

,year,month,temperature,humidity,air_pressure,monthly_rainfall,latitude,longitude,oni_index,el_nino,la_nina
count,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.00000,4.800000e+01,48.000000,48.000000,48.000000
mean,2021.500000,6.500000,27.303958,86.433414,1009.316807,175.656250,10.53333,1.228333e+02,-0.222917,0.166667,0.416667
std,1.129865,3.488583,0.695420,2.916370,0.914146,113.197641,0.00000,1.436124e-14,0.957443,0.376622,0.498224
min,2020.000000,1.000000,25.861290,76.900000,1007.845161,6.900000,10.53333,1.228333e+02,-1.600000,0.000000,0.000000
25%,2020.750000,3.750000,27.024654,85.050000,1008.773978,79.000000,10.53333,1.228333e+02,-0.900000,0.000000,0.000000
50%,2021.500000,6.500000,27.248387,86.753226,1009.095161,177.100000,10.53333,1.228333e+02,-0.500000,0.000000,0.000000
75%,2022.250000,9.250000,27.517930,88.620968,1009.694167,260.750000,10.53333,1.228333e+02,0.100000,0.000000,1.000000
max,2023.000000,12.000000,29.516667,90.935484,1012.544828,435.900000,10.53333,1.228333e+02,2.100000,1.000000,1.000000


In [36]:
# Drop missing values
initial_count = len(df_monthly)
df_monthly = df_monthly.dropna()

In [37]:
df_monthly.describe()

,year,month,temperature,humidity,air_pressure,monthly_rainfall,latitude,longitude,oni_index,el_nino,la_nina
count,8868.000000,8868.000000,8868.000000,8868.000000,8868.000000,8868.000000,8868.000000,8868.000000,8868.000000,8868.000000,8868.000000
mean,2022.139378,6.286874,26.740416,88.986924,1003.247961,233.494452,12.014342,122.675400,-0.122767,0.190347,0.333784
std,1.519756,3.506976,1.531093,4.961026,18.810297,163.238752,3.135951,1.793293,0.917704,0.392597,0.471590
min,2020.000000,1.000000,15.829032,66.580645,850.890000,0.300000,6.087220,118.733330,-1.600000,0.000000,0.000000
25%,2021.000000,3.000000,26.187097,86.483871,1005.316667,107.600000,9.515280,121.025398,-0.700000,0.000000,0.000000
50%,2022.000000,6.000000,26.903280,90.225806,1008.473333,217.000000,11.775280,122.801109,-0.300000,0.000000,0.000000
75%,2023.000000,9.000000,27.503226,92.483037,1009.864343,323.425000,14.570100,123.983330,0.200000,0.000000,1.000000
max,2025.000000,12.000000,31.676667,99.838710,1016.424138,1689.500000,18.198891,126.517502,2.100000,1.000000,1.000000


In [ ]:
# Drop missing values
initial_count = len(df_monthly)
df_monthly = df_monthly.dropna()


# Keep only cities with complete 48 months
city_counts = df_monthly.groupby('city').size()
complete_cities = city_counts[city_counts == NUM_MONTHS].index
df_monthly = df_monthly[df_monthly['city'].isin(complete_cities)]

print(f"Monthly records: {len(df_monthly)}")
print(f"Cities with complete data: {len(complete_cities)}")
print(f"Features: {len(df_monthly.columns) - 4}")

Monthly records: 8316
Cities with complete data: 132
Features: 8
